In [1]:
import h5py
import matplotlib.pyplot as plt
import numpy as np
from EMG_Step_Extr_Pipeline import EMGProcessor
import pandas as pd

from Data_Segmentation import Data_Segmentor
from Step_Extraction_Stats import Step_Extraction_Stats
import math
from datetime import datetime


In [2]:
defaultDelsysAttachments = ['TAL', 'TAR', 'LGL', 'LGR', 'VLL', 'VLR', 'BFL', 'BFR','None', 'None', 'None', 'None', 'None', 'LK', 'RK']
#Trials 1-2 are test trials
# Trials 4-7 are real trials
# Trial 6 is a dud
Trial4File = h5py.File("Trial 4.h5")
Trial5File = h5py.File("Trial 5.h5")
Trial6File = h5py.File("Trial 6.h5")
Trial7File = h5py.File("Trial 7.h5")



In [3]:
Trial4File["Sensor 14/Analog 1"].attrs["SampleRate"]

296.2962962962963

In [3]:
# Data Segmentation for Trial 4
dataSegmTrial4 = Data_Segmentor()

dataSegmTrial4.parseData(Trial4File)

dataSegmTrial4.filt_rect_smooth_EMG()

rawEMGdf4 = dataSegmTrial4.getEMGData()
filtEMGdf4 = dataSegmTrial4.get_filt_rect_smooth_emg()

dataSegmTrial4.bin_pressure_data()
binnedPressData4 = dataSegmTrial4.get_binned_pressure_data()

In [4]:
# Data Segmentation for Trial 5
dataSegmTrial5 = Data_Segmentor()

dataSegmTrial5.parseData(Trial5File)

dataSegmTrial5.filt_rect_smooth_EMG()

rawEMGdf5 = dataSegmTrial5.getEMGData()
filtEMGdf5 = dataSegmTrial5.get_filt_rect_smooth_emg()

dataSegmTrial5.bin_pressure_data()
binnedPressData5 = dataSegmTrial5.get_binned_pressure_data()

In [5]:
# Data Segmentation for Trial 7
dataSegmTrial7 = Data_Segmentor()

dataSegmTrial7.parseData(Trial7File)

dataSegmTrial7.filt_rect_smooth_EMG()

rawEMGdf7 = dataSegmTrial7.getEMGData()
filtEMGdf7 = dataSegmTrial7.get_filt_rect_smooth_emg()

dataSegmTrial7.bin_pressure_data()
binnedPressData7 = dataSegmTrial7.get_binned_pressure_data()

rawGonDf4 = dataSegmTrial4.getGonData()


# Extracting All Transition Time Stamps for Each Trial

In [6]:
trial4Transitions = [datetime.fromtimestamp(Trial4File.attrs["startTime"]), datetime.fromtimestamp(1710876473.60037), datetime.fromtimestamp(1710876484.905785), 
                     datetime.fromtimestamp(1710876494.747219), datetime.fromtimestamp(1710876501.706223), datetime.fromtimestamp(1710876515.63894), datetime.fromtimestamp(Trial4File.attrs["stopTime"])]

trial5Transitions = [datetime.fromtimestamp(Trial5File.attrs["startTime"]), datetime.fromtimestamp(1710876568.206406), datetime.fromtimestamp(1710876581.310392), 
                     datetime.fromtimestamp(1710876589.748959), datetime.fromtimestamp(1710876597.072555), datetime.fromtimestamp(1710876610.109591), datetime.fromtimestamp(Trial5File.attrs["stopTime"])]

trial7Transitions = [datetime.fromtimestamp(Trial7File.attrs["startTime"]), datetime.fromtimestamp(1710876906.509646), datetime.fromtimestamp(1710876918.805888), 
                     datetime.fromtimestamp(1710876927.801345), datetime.fromtimestamp(1710876935.719349), datetime.fromtimestamp(1710876948.398316), datetime.fromtimestamp(Trial7File.attrs["stopTime"])]

In [7]:
secTimeDiffDict = {}

transArr = [trial4Transitions, trial5Transitions, trial7Transitions]
nameArr = ["trial4", "trial5", "trial7"]
iterator = 0

# Extracting the time difference in seconds 
for arr in transArr:
    secTimeDiffDict[nameArr[iterator]] = []
    for x in range(len(arr) - 1):
        timedelta = arr[x + 1] - arr[0]
        # Return the total time difference in seconds
        
        secTimeDiffDict[nameArr[iterator]].append(timedelta.days * 24 * 3600 + timedelta.seconds)

    iterator += 1


secTimeDiffDict

{'trial4': [9, 20, 30, 37, 51, 65],
 'trial5': [8, 21, 30, 37, 50, 62],
 'trial7': [9, 21, 30, 38, 51, 61]}

In [8]:
emgSampleRate = Trial4File["Sensor 1/EMG 1"].attrs["SampleRate"]
emgSampleRate = np.round(emgSampleRate).astype(int)

emgSampleRate

# Changing all second timestamps to emg data indexes
for key in secTimeDiffDict.keys():
    arr = secTimeDiffDict[key]
    for x in range(len(arr)):
        arr[x] *= emgSampleRate
        
    secTimeDiffDict[key] = arr

secTimeDiffDict

{'trial4': [17334, 38520, 57780, 71262, 98226, 125190],
 'trial5': [15408, 40446, 57780, 71262, 96300, 119412],
 'trial7': [17334, 40446, 57780, 73188, 98226, 117486]}

# Extracting EMG Data Based Upon Time Diff Indexes

In [13]:
terrains = ["flat1", "uneven1", "upstairs", "downstairs", "uneven2", "flat2"]

outputDict = {}
inputDict = secTimeDiffDict
inputDf = filtEMGdf4
inputTransInds = secTimeDiffDict["trial4"]

# 
for x in range(len(terrains)):
    # Getting flat terrain data at the end of the trial
    if (x == len(terrains) - 1):
        outputDict[terrains[x]] = inputDf.iloc[inputTransInds[x - 1]:len(inputDf)]
        outputDict[terrains[x]].index = np.arange(len(outputDict[terrains[x]]))

    # Getting flat terrains data from the beginning of the trial
    elif (x == 0):
        outputDict[terrains[x]] = inputDf.iloc[0:inputTransInds[x]]
        outputDict[terrains[x]].index = np.arange(len(outputDict[terrains[x]]))

    else:
        outputDict[terrains[x]] = inputDf.iloc[inputTransInds[x-1]:inputTransInds[x]]
        outputDict[terrains[x]].index = np.arange(len(outputDict[terrains[x]]))

outputDict.keys()


dict_keys(['flat1', 'uneven1', 'upstairs', 'downstairs', 'uneven2', 'flat2'])

In [14]:
# Saving the parsed data
trialName = "trial4"
for key in outputDict.keys():
    saveName = trialName + "_" + key + "_" + "emg" + ".csv"
    outputDict[key].to_csv(saveName, index = False)


# Extracting XSensor Data for Each Terrain

In [15]:
# Have to use sampleRatio to remap the indices since the XSensor doesn't seem to actually be sampling at 10 Hz
sampleRatio = len(binnedPressData4) / len(rawEMGdf4)

pressIndDict = {}

# Changing all second timestamps to emg data indexes
for key in secTimeDiffDict.keys():
    arr = secTimeDiffDict[key]
    for x in range(len(arr)):
        val = arr[x] *sampleRatio
        arr[x] = np.round(val).astype(int)
        
    pressIndDict[key] = arr

pressIndDict

{'trial4': [56, 125, 188, 231, 319, 406],
 'trial5': [50, 131, 188, 231, 313, 388],
 'trial7': [56, 131, 188, 238, 319, 381]}

In [20]:
terrains = ["flat1", "uneven1", "upstairs", "downstairs", "uneven2", "flat2"]

outputDict = {}
inputDf = binnedPressData7
inputTransInds = pressIndDict["trial7"]

# 
for x in range(len(terrains)):
    # Getting flat terrain data at the end of the trial
    if (x == len(terrains) - 1):
        outputDict[terrains[x]] = inputDf.iloc[inputTransInds[x - 1]:len(inputDf)]
        outputDict[terrains[x]].index = np.arange(len(outputDict[terrains[x]]))

    # Getting flat terrains data from the beginning of the trial
    elif (x == 0):
        outputDict[terrains[x]] = inputDf.iloc[0:inputTransInds[x]]
        outputDict[terrains[x]].index = np.arange(len(outputDict[terrains[x]]))

    else:
        outputDict[terrains[x]] = inputDf.iloc[inputTransInds[x-1]:inputTransInds[x]]
        outputDict[terrains[x]].index = np.arange(len(outputDict[terrains[x]]))

outputDict.keys()

dict_keys(['flat1', 'uneven1', 'upstairs', 'downstairs', 'uneven2', 'flat2'])

In [21]:
# Saving the parsed data
trialName = "trial7"
for key in outputDict.keys():
    saveName = trialName + "_" + key + "_" + "press" + ".csv"
    outputDict[key].to_csv(saveName, index = False)

In [107]:
print(sampleRatio)
10/1926

0.0032460099424385137


0.005192107995846314